In [ ]:
# import statements...
import pandas as pd
import numpy as np
import ast

In [ ]:
from google.colab import drive
drive.mount('/content/drive'),
# filepaths for data:
parks_df = pd.read_csv('/content/drive/My Drive/CIS450/450/National Parks Datasets/biodiversity/parks.csv')
species_df = pd.read_csv('/content/drive/My Drive/CIS450/450/National Parks Datasets/biodiversity/species.csv')
trails_df = pd.read_csv('/content/drive/My Drive/CIS450/450/National Parks Datasets/AllTrails data - nationalpark.csv')
us_cities_df = pd.read_csv('/content/drive/My Drive/CIS450/450/National Parks Datasets/uscities.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
# species — drop record status, seasonality, nativeness, occurrence
# species_df = species_df.drop(columns=['Record Status', 'Seasonality', 'Nativeness', 'Occurrence'])

In [ ]:
# species — replace nulls in abundance with unknown, conservation status with not endangered
species_df['Abundance'] = species_df['Abundance'].fillna('Unknown')
species_df['Nativeness'] = species_df['Nativeness'].fillna('Unknown')
species_df['Occurrence'] = species_df['Occurrence'].fillna('Unknown')
species_df['Conservation Status'] = species_df['Conservation Status'].fillna('Not Endangered')

In [ ]:
# standarize column names in parks
parks_df = parks_df.rename(columns={'Park Code': 'park_code', 'Park Name': 'park_name', 'State': 'state', 'Acres': 'acres', 'Latitude': 'lat', 'Longitude': 'lng'})

In [ ]:
# standarize column names in parks
species_df = species_df.rename(columns={'Species ID': 'species_id', 'Park Name': 'park_name', 'Category': 'category', 'Order': 'order', 'Family': 'family', 'Scientific Name': 'scientific_name', 'Common Names': 'common_names', 'Abundance': 'abundance', 'Conservation Status': 'conservation_status', 'Record Status': 'record_status', 'Seasonality': 'seasonality', 'Nativeness': 'nativeness', 'Occurrence': 'occurrence'})

In [ ]:
species_df

,species_id,park_name,category,order,family,scientific_name,common_names,record_status,occurrence,nativeness,abundance,seasonality,conservation_status,Unnamed: 13
0,ACAD-1000,Acadia National Park,Mammal,Artiodactyla,Cervidae,Alces alces,Moose,Approved,Present,Native,Rare,Resident,Not Endangered,NaN
1,ACAD-1001,Acadia National Park,Mammal,Artiodactyla,Cervidae,Odocoileus virginianus,"Northern White-Tailed Deer, Virginia Deer, Whi...",Approved,Present,Native,Abundant,NaN,Not Endangered,NaN
2,ACAD-1002,Acadia National Park,Mammal,Carnivora,Canidae,Canis latrans,"Coyote, Eastern Coyote",Approved,Present,Not Native,Common,NaN,Species of Concern,NaN
3,ACAD-1003,Acadia National Park,Mammal,Carnivora,Canidae,Canis lupus,"Eastern Timber Wolf, Gray Wolf, Timber Wolf",Approved,Not Confirmed,Native,Unknown,NaN,Endangered,NaN
4,ACAD-1004,Acadia National Park,Mammal,Carnivora,Canidae,Vulpes vulpes,"Black Fox, Cross Fox, Eastern Red Fox, Fox, Re...",Approved,Present,Unknown,Common,Breeder,Not Endangered,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119243,ZION-2791,Zion National Park,Vascular Plant,Solanales,Solanaceae,Solanum triflorum,Cut-Leaf Nightshade,Approved,Present,Native,Uncommon,NaN,Not Endangered,NaN
119244,ZION-2792,Zion National Park,Vascular Plant,Vitales,Vitaceae,Vitis arizonica,Canyon Grape,Approved,Present,Native,Uncommon,NaN,Not Endangered,NaN
119245,ZION-2793,Zion National Park,Vascular Plant,Vitales,Vitaceae,Vitis vinifera,Wine Grape,Approved,Present,Not Native,Uncommon,NaN,Not Endangered,NaN
119246,ZION-2794,Zion National Park,Vascular Plant,Zygophyllales,Zygophyllaceae,Larrea tridentata,Creosote Bush,Approved,Present,Native,Rare,NaN,Not Endangered,NaN


In [ ]:
species_df = species_df.drop(columns=['Unnamed: 13'])

In [ ]:
# allTrails — rename area_name to park_name
trails_df = trails_df.rename(columns={"area_name": "park_name"})

In [ ]:
# allTrails — change states to state abbreviations
us_state_abbrev = {
'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO',
'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID',
'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA',
'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ',
'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD',
'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA',
'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'}

trails_df['state_name'] = trails_df['state_name'].map(us_state_abbrev).fillna(trails_df['state_name'])

# allTrails — work around Hawaii
trails_df.loc[trails_df['country_name'] == 'Hawaii', 'state_name'] = 'HI'

In [ ]:
# allTrails — drop country_name
trails_df = trails_df.drop(columns=['country_name', 'visitor_usage'])

In [ ]:
# allTrails — split up activities
trails_activities_df = pd.DataFrame(columns=['trail_id', 'activity'])

for index, row in trails_df.iterrows():
	trail_id, activity_list = row['trail_id'], ast.literal_eval(row['activities'])
	for activity in activity_list:
		new_entry = {'trail_id': trail_id, 'activity': activity}
		trails_activities_df = trails_activities_df.append(new_entry, ignore_index=True)

In [ ]:
trails_activities_df

,trail_id,activity
0,10020048,birding
1,10020048,camping
2,10020048,hiking
3,10020048,nature-trips
4,10020048,trail-running
...,...,...
13290,10236001,hiking
13291,10258707,hiking
13292,10258707,walking
13293,10014989,hiking


In [ ]:
# allTrails — split up features
trails_features_df = pd.DataFrame(columns=['trail_id', 'feature'])

for index, row in trails_df.iterrows():
	trail_id, feature_list = row['trail_id'], ast.literal_eval(row['features'])
	for feature in feature_list:
		new_entry = {'trail_id': trail_id, 'feature': feature}
		trails_features_df = trails_features_df.append(new_entry, ignore_index=True)

In [ ]:
trails_features_df

,trail_id,feature
0,10020048,dogs-no
1,10020048,forest
2,10020048,river
3,10020048,views
4,10020048,waterfall
...,...,...
16328,10258707,views
16329,10014989,dogs-no
16330,10014989,views
16331,10014989,wildlife


In [ ]:
trails_df = trails_df.drop(columns=['activities'])

In [ ]:
trails_df = trails_df.drop(columns=['features'])

In [ ]:
# allTrails — split up _geoloc
trails_df['lat'] = None
trails_df['lng'] = None
for index, row in trails_df.iterrows():
  trail_id, loc_dict = row['trail_id'], ast.literal_eval(row['_geoloc'])
  lat, lng = loc_dict['lat'], loc_dict['lng']
  trails_df.loc[trails_df['trail_id']==trail_id, 'lat'] = lat
  trails_df.loc[trails_df['trail_id']==trail_id, 'lng'] = lng
trails_df = trails_df.drop(columns=['_geoloc'])

In [ ]:
trails_df

,trail_id,name,park_name,city_name,state_name,popularity,length,elevation_gain,difficulty_rating,route_type,avg_rating,num_reviews,units,lat,lng
0,10020048,Harding Ice Field Trail,Kenai Fjords National Park,Seward,AK,24.8931,15610.598,1161.8976,5,out and back,5.0,423,i,60.1885,-149.632
1,10236086,Mount Healy Overlook Trail,Denali National Park,Denali National Park,AK,18.0311,6920.162,507.7968,3,out and back,4.5,260,i,63.7305,-148.92
2,10267857,Exit Glacier Trail,Kenai Fjords National Park,Seward,AK,17.7821,2896.812,81.9912,1,out and back,4.5,224,i,60.1888,-149.631
3,10236076,Horseshoe Lake Trail,Denali National Park,Denali National Park,AK,16.2674,3379.614,119.7864,1,loop,4.5,237,i,63.7366,-148.915
4,10236082,Triple Lakes Trail,Denali National Park,Denali National Park,AK,12.5935,29772.790,1124.7120,5,out and back,4.5,110,i,63.7332,-148.897
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3308,10008302,Silversword Loop Via Halemau'u Trail,Haleakala National Park,Kula,HI,9.3861,20116.750,1105.8144,5,loop,4.5,43,m,20.7527,-156.229
3309,10236001,Keonehe'ehe'e Trail,Haleakala National Park,Kula,HI,9.1555,28324.384,1171.9560,5,out and back,5.0,22,m,20.7145,-156.251
3310,10258707,Red Hill Overlook Summit Trail,Haleakala National Park,Kula,HI,8.5066,321.868,3.9624,1,out and back,4.5,31,m,20.7101,-156.254
3311,10014989,Kaupo Trail,Haleakala National Park,Kula,HI,8.3240,19312.080,1670.9136,5,out and back,4.0,8,m,20.6498,-156.137


In [ ]:
# cities — drop city_ascii, zips, ranking, timezone, incorporated, military, county_name, county_fips, source
us_cities_df = us_cities_df.drop(columns=['city_ascii', 'zips', 'ranking', 'timezone', 'incorporated', 'military', 'county_name', 'county_fips', 'source'])

In [ ]:
# cities — drop rows with population 0
us_cities_df = us_cities_df[us_cities_df['population'] != 0]

In [ ]:
us_cities_df

,city,state_id,state_name,lat,lng,population,density,id
0,New York,NY,New York,40.6943,-73.9249,18713220.0,10715.0,1840034016
1,Los Angeles,CA,California,34.1139,-118.4068,12750807.0,3276.0,1840020491
2,Chicago,IL,Illinois,41.8373,-87.6862,8604203.0,4574.0,1840000494
3,Miami,FL,Florida,25.7839,-80.2102,6445545.0,5019.0,1840015149
4,Dallas,TX,Texas,32.7936,-96.7662,5743938.0,1526.0,1840019440
...,...,...,...,...,...,...,...,...
29272,Poso Park,CA,California,35.8114,-118.6364,2.0,17.0,1840024878
29273,Lotsee,OK,Oklahoma,36.1334,-96.2091,2.0,39.0,1840021674
29274,The Ranch,MN,Minnesota,47.3198,-95.6952,2.0,2.0,1840039629
29275,Roswell,SD,South Dakota,44.0060,-97.6975,2.0,1.0,1840002541


In [ ]:
parks_df.to_csv(path_or_buf='/content/drive/My Drive/CIS450/450/Processed Data/parks.csv', index=False)
species_df.to_csv(path_or_buf='/content/drive/My Drive/CIS450/450/Processed Data/species.csv', index=False)
trails_df.to_csv(path_or_buf='/content/drive/My Drive/CIS450/450/Processed Data/trails.csv', index=False)
trails_activities_df.to_csv(path_or_buf='/content/drive/My Drive/CIS450/450/Processed Data/trails_activities.csv', index=False)
trails_features_df.to_csv(path_or_buf='/content/drive/My Drive/CIS450/450/Processed Data/trails_features.csv', index=False)
us_cities_df.to_csv(path_or_buf='/content/drive/My Drive/CIS450/450/Processed Data/us_cities.csv', index=False)